In [46]:
!pip install langchain
!pip install pinecone-client==2.2.2
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate transformers

In [1]:
!pip install huggingface_hub==0.16.4

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA


In [4]:
!mkdir pdfs


mkdir: cannot create directory ‘pdfs’: File exists


In [5]:
!pip install pypdf

In [6]:
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [7]:
print(data)

[Document(page_content='TOP FEATURES\n• Picture: 4K Glo Panel  I400 Nits Brightness  I94% NTSC Color Gamut IDolby Vision IQ  IHDR10 \nPlus, HLG  IMotion Enhancement, MEMC IActive Contrast  IGame Mode  IALLM ( Auto Low \nLatency Mode )IVRR ( Variable Refresh Rate ) \n• Sound: Built -inDJ Subwoofer I  104watt DJ sound I4xMaster speakers + 1x Sub -woofer  IDolby \nAtmos Sound Enhancement  ISub-Woofer Bass Boost Control   IAudio Only Mode\n• Content and Software: Google TV OS  IAdvanced Cricket Mode  ICinema Mode  IAI Picture Engine  \nIGoogle Play Store  INetflix, Prime Video, YouTube & Hotstar Licensed Apps on Remote  IWishlist  I\nLibrary  IKids Mode with Advanced Parental Lock  IDevice Assistant App\n• Hardware: Glo AI Processor IFar-Field Microphone for Hands -Free Voice Search IAmbient Light \nSensor  IVu ActiVoice Remote Control  IHigh speed, 16GB eMMC 5.1 storage I2GB RAM    \n• Connectivity :Google Chrome Cast  IBluetooth 5.1  I2.4/5GHz WiFiIHDMI CEC & eARC /ARC  I\nHDMI x 3no’s  

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [9]:
docs = text_splitter.split_documents(data)

In [10]:
print(len(docs))

137


In [11]:
docs[3]

Document(page_content='DISPLAY\nScreen Size (inch) cm (65)164cm, (55)139cm,  (50)126cm\nModel Number 65GloLED  I  55GloLED  I 50GloLED  \nResolution & Backlight Source 3840  x 2160, DLED\nPanel Technology Glo Panel\nBrightness 400 Nits\nPanel Refresh Rate 60Hz\nWide Color Gamut Yes\nColour Depth 10 Bit\nHigh Dynamic Range Dolby Vision IQ, HDR10+, HLG\nColor Volume 94% NTSC\nActive Contrast Yes\nGame Feature Game Mode, ALLM, VRR\nMEMC YesAudio Output 104 watt Speakers\nNumber of Speakers Master x 4  ISub-woofer x 1', metadata={'source': 'pdfs/vu tv.pdf', 'page': 1})

In [12]:
docs[4]

Document(page_content='Sub-Woofer Control Yes\nDolby Audio MS12, Dolby, Dolby Digital, \nDolby Digital Plus\nAudio Enhancement Type Dolby Atmos\nAudio Only Mode Yes\nExternal Speaker Connection TV/ eARC / ARC/ SPDIF/ Bluetooth\n4K @ 60Hz Yes\nHDMI version HDMI 2.1\nProcessor Glo AI Processor\nStorage / DDR RAM 16GB / 2GB\nGPU Yes\nGame Mode Yes\nALLM Yes\nVRR Yes\nDimensions with stand \n(WxHxD ) in mm(65) 1447 ×907 ×345\n(55) 1230 ×773 ×305\n(50) 1116 ×710 ×305\nDimensions without stand \n(WxHxD ) in mm(65) 1447 ×839 ×76', metadata={'source': 'pdfs/vu tv.pdf', 'page': 1})

In [13]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [14]:
query_result = embeddings.embed_query("Hello World")

In [15]:
print("Length", len(query_result))


Length 384


In [16]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'GET YOUR API KEY FROM PINECONE')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'GET YOUR ENVIRONMENT FROM PINECONE')




In [ ]:
!pip install pinecone_client==2.2.2

In [17]:
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchainpinecone" # put in the name of your pinecone index here

In [18]:
docsearch=Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)


In [19]:
query = "Suggest me a laptop at $500"

In [20]:
docs = docsearch.similarity_search(query, k=4)

In [21]:
docs

[Document(page_content='Laptop\nNAME\n:\nAsus\nvivobook\n16\nLaptop\nPrice\n:\n65,000\nLaptop\nRam\n:\n16\ngb\nLaptop\nHDD\n:\n512\ngb'),
 Document(page_content='Laptop\nNAME\n:\nAsus\nvivobook\n16\nLaptop\nPrice\n:\n65,000\nLaptop\nRam\n:\n16\ngb\nLaptop\nHDD\n:\n512\ngb'),
 Document(page_content='Laptop\nNAME\n:\nAsus\nvivobook\n16\nLaptop\nPrice\n:\n65,000\nLaptop\nRam\n:\n16\ngb\nLaptop\nHDD\n:\n512\ngb'),
 Document(page_content='Laptop\nNAME\n:\nAsus\nvivobook\n16\nLaptop\nPrice\n:\n65,000\nLaptop\nRam\n:\n16\ngb\nLaptop\nHDD\n:\n512\ngb')]

In [22]:
!pip install huggingface_hub==0.16.4

In [23]:
from huggingface_hub import notebook_login
import torch

In [24]:
notebook_login()

In [25]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
pip install accelerate bitsandbytes

In [26]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,

                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [27]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )



In [28]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

In [29]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""

In [30]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"

In [31]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [32]:
instruction = """
{context}

Question: {question}
"""

In [33]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [34]:
template

"[INST]<>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [35]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [36]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [37]:
result = qa_chain("Best laptop with 8gb Ram")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [38]:
result['result']

'  Based on the information provided, the best laptop with 8GB RAM from the HP 14/15 Notebook series is the HP 15 Notebook series.\n\nThe HP 15 Notebook series offers the following specifications:\n\n* Processor: Intel® Celeron® N3050 Processor (1.6GHz, up to 2.16GHz, 2MB cache, 2 cores)\n* System Memory: 2GB DDR3L-1600 SDRAM (1 DIMM); 500GB (5400RPM) hard drive\n* Display: 14.0/15.6-inch diagonal HD BrightView LED-backlit Display (1366x768)\n* Operating System: Windows 10 (64-bit)\n\nAll of these specifications are available with 8GB RAM, making the HP 15 Notebook series the best option for someone looking for a laptop with 8GB RAM from the HP 14/15 Notebook series.'

In [ ]:
while True:
    user_input=input(f"prompt:")
    if user_input=='exit':
        print('Exiting')
        sys.exit()
    if user_input=='':
        continue
    result=qa_chain({'query':user_input})
    print(f"Answer:{result['result']}")

prompt:Best TV to buy now
Answer:  Based on the information provided, the best TV to buy now depends on several factors such as budget, screen size, and personal preferences. Here are some recommendations for each of the three TV models mentioned:

Sony BRA VIA KD-65X80K: If you have a budget of ₹94,990 or more and are looking for a large screen TV with a high refresh rate, this model might be the best option. It has a 65-inch screen, 4K resolution, and a 50 Hz refresh rate, which is ideal for gaming and fast-paced content.

Sony BRA VIA KD-50X80L: If you have a budget of ₹75,990 or more and are looking for a smaller screen size, this model might be the best option. It has a 50-inch screen, 4K resolution, and a 50 Hz refresh rate, which is suitable for gaming and general viewing.

Sony BRA VIA KD-77X80K: If you have a budget of ₹94,990 or more and are looking for an even larger screen, this model might be the best option. It has a 77-inch screen, 4K resolution, and a 50 Hz refresh rate

In [ ]:
base_model = AutoModelForCasualLM.from_pretrained(
    model_name,
    return_dict = True,
    torch_dtype = torch.float16,
    device_amp = device_map
)

In [ ]:
dtype